In [2]:
import pandas as pd
import xarray as xr
import os
import glob

if __name__ == '__main__':
    _dir = os.getcwd()
    _loc_files = glob.glob(_dir + '/CRE_data/*.nc')
print(_loc_files)

print('done')

['/home/jupyter/cae-notebooks/CRE_data/Minimum air temperature at 2m_GWL1.0_50th.nc', '/home/jupyter/cae-notebooks/CRE_data/Maximum air temperature at 2m_GWL1.5_50th.nc', '/home/jupyter/cae-notebooks/CRE_data/Precipitation (total)_GWL1.0_50th.nc', '/home/jupyter/cae-notebooks/CRE_data/Precipitation (total)_GWL2.0_90th.nc', '/home/jupyter/cae-notebooks/CRE_data/Precipitation (total)_GWL1.5_50th.nc', '/home/jupyter/cae-notebooks/CRE_data/Minimum air temperature at 2m_GWL2.0_50th.nc', '/home/jupyter/cae-notebooks/CRE_data/Minimum air temperature at 2m_GWL2.0_90th.nc', '/home/jupyter/cae-notebooks/CRE_data/Maximum air temperature at 2m_GWL2.0_50th.nc', '/home/jupyter/cae-notebooks/CRE_data/Minimum air temperature at 2m_GWL1.5_50th.nc', '/home/jupyter/cae-notebooks/CRE_data/Maximum air temperature at 2m_GWL2.0_90th.nc', '/home/jupyter/cae-notebooks/CRE_data/Minimum air temperature at 2m_GWL1.5_90th.nc', '/home/jupyter/cae-notebooks/CRE_data/Precipitation (total)_GWL1.5_90th.nc', '/home/jupy

In [4]:
ds = xr.open_dataset('/home/jupyter/cae-notebooks/CRE_data/Precipitation (total)_GWL2.0_90th.nc')
print(ds)

<xarray.Dataset> Size: 3MB
Dimensions:                        (lat: 171, lon: 196, month: 12)
Coordinates:
    warming_level                  float64 8B ...
  * lat                            (lat) float32 684B 33.17 33.2 ... 38.45 38.48
  * lon                            (lon) float32 784B -120.2 -120.2 ... -114.1
  * month                          (month) float64 96B 1.0 2.0 3.0 ... 11.0 12.0
    quantile                       float64 8B ...
Data variables:
    __xarray_dataarray_variable__  (month, lat, lon) float64 3MB ...


In [3]:

# Dictionary to shorten the nc file name prefixes
prefix_dict = {
    "Maximum air temperature at 2m": "MaxT",
    "Minimum air temperature at 2m": "MinT",
    "Precipitation (total)": "Prc",
    "Maximum wind speed at 10m": "MaxW"
}

# Function to convert nc file to a dictionary of DataFrames and sheet name prefix
def nc_to_dfs(nc_file):
    ds = xr.open_dataset(nc_file)
    file_name = os.path.splitext(os.path.basename(nc_file))[0]
    
    # Find the matching prefix and shorten it
    for long_prefix, short_prefix in prefix_dict.items():
        if file_name.startswith(long_prefix):
            sheet_name_prefix = file_name.replace(long_prefix, short_prefix)
            break
    else:
        sheet_name_prefix = file_name  # Use the full name if no match is found
    
    dfs = {sheet_name_prefix: ds.to_dataframe().dropna().reset_index()}
    return dfs

# Function to add gridcode to DataFrames based on matching lat, lon
def add_gridcode_to_dfs(dfs_list, gridcode_df):
    for dfs in dfs_list:
        for sheet_name, df in dfs.items():
            df_with_gridcode = pd.merge(df, gridcode_df, on=['lat', 'lon'], how='right')
            dfs[sheet_name] = df_with_gridcode
    return dfs_list

# Function to save grouped DataFrames to separate Excel files
def save_grouped_dfs_to_excel(dfs_grouped):
    for prefix, dfs_list in dfs_grouped.items():
        with pd.ExcelWriter(f"{prefix}.xlsx", engine='xlsxwriter') as writer:
            for dfs in dfs_list:
                for sheet_name, df in dfs.items():
                    df.to_excel(writer, sheet_name=sheet_name[:31], index=False)  # Excel sheet names must be <= 31 characters

# Example usage
nc_files = _loc_files  # Replace with your .nc file paths

# Read the CSV file with lat, lon, and gridcode
gridcode_csv = '~/cae-notebooks/SCE_ST_LOCA_LatLon_Gridcodes.csv'
gridcode_df = pd.read_csv(gridcode_csv)

# Group DataFrames by prefix
dfs_grouped = {short_prefix: [] for short_prefix in prefix_dict.values()}
for nc_file in nc_files:
    dfs = nc_to_dfs(nc_file)
    dfs_list_with_gridcode = add_gridcode_to_dfs([dfs], gridcode_df)
    for sheet_name in dfs.keys():
        prefix = sheet_name.split('_')[0]
        dfs_grouped[prefix].append(dfs_list_with_gridcode[0])

# Save grouped DataFrames to separate Excel files
save_grouped_dfs_to_excel(dfs_grouped)

# print(f"Converted {nc_files} to separate Excel files for each variable and added gridcodes.")